### Import packages

In [1]:
import numpy as np
import copy
import matplotlib.pyplot as plt
from scipy.spatial.distance import squareform, pdist, cdist
import scipy.linalg as lng
from numpy.linalg import slogdet
from sklearn.metrics import r2_score, mean_squared_error
import scipy.optimize as opt
import time
import logging

### Download sample data

In [2]:
# download sample data
!git clone https://github.com/sjliu68/pyMRAsampleData.git

'git' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


### Base functions for calculating distance

In [ ]:
#%% base funtions for calculating D
def dist(locs, locs2=np.array([]), circular=False):
    locs = locs if np.ndim(locs)==2 else np.reshape(locs, [len(locs), 1])
    if circular:
        if len(locs2):
            xv, yv = np.meshgrid(locs, locs2)
        else:
            xv, yv = np.meshgrid(locs, locs)
        m = np.minimum(xv, yv)
        M = np.maximum(xv, yv)
        dist = np.matrix(np.minimum(M - m, m + 1-M).T)
    else:
        if len(locs2):
            dist = np.matrix(cdist(locs, locs2))
        else:
            dist = np.matrix(squareform(pdist(locs)))
    return dist

def GetD(locs, locs2=np.array([]), circular=False):
    D = dist(locs, locs2, circular)
    return(D)

### Load dataset and make a small region with 80% missing.

In [ ]:
#%% load dataset
x1 = np.load('pyMRAsampleData/xlon2019.npy')
x2 = np.load('pyMRAsampleData/xlat2019.npy')
mean = np.load('pyMRAsampleData/mean2019.npy')   
mean -= 97
mean /= 10

imx1,imx2,imy1,imy2 = 670,770,190,260

data = mean[imx1:imx2,imy1:imy2]
data2 = copy.deepcopy(mean[imx1:imx2,imy1:imy2])
plt.matshow(data)
plt.show()


x1 = x1[imx1:imx2,imy1:imy2]
x2 = x2[imx1:imx2,imy1:imy2]
x1 -= x1.min()
x2 -= x2.min()
x1 /= x1.max()
x2 /= x2.max()


### make random train/test
np.random.seed(42)
_x = 50
_y = 20
n = 30
data[_x:_x+n,_y:_y+n] = np.nan
for i in range(n):
    for j in range(n):
        if np.random.rand()<0.2:
            data[_x+i,_y+j] = data2[_x+i,_y+j]

plt.matshow(data)
plt.show()

#%
idx = np.isnan(data)
train_y = data[~idx]
train_x1 = x1[~idx]
train_x2 = x2[~idx]


test_x1 = x1[idx]
test_x2 = x2[idx]
test_y = data2[idx]
test_X = np.asarray([test_x1,test_x2]).T
train_X = np.asarray([train_x1,train_x2]).T

idx2 = np.isnan(test_y)
test_y2 = test_y[~idx2]

### overwrite here, make all data including nan
train_y2 = data.reshape(-1,1)
idx = np.arange(train_y.shape[0])
train_X2 = np.asarray([x1.reshape(-1),x2.reshape(-1)]).T

train_y # without NaN
train_X # without NaN

## log-likelihood function
#### inputs: 
    YQ, AOD [N1,1], N1=6297
    XQ, ones vector [N1,1]
    D, distance matrix [N1,N1]
    beta, scalar
    var, scalar
    phi, scalar
    tau2, scalar

#### outputs:
    negative of the log-likelihood
    



## general procedures
$$
K = \sigma^2 \cdot \exp(- \phi \cdot D) + \tau^2 \cdot I_{N_1} \\
u = X(Q)\beta \\
l(u, K) = \frac{1}{2} logdet(K) - \frac{1}{2} (y(Q)-u)^T K^{-1} (y(Q)-u) \\ 
$$
$$
v = (y(Q)-u) \\
l(u, K) = \frac{1}{2} logdet(K) - \frac{1}{2} v^T K^{-1} v \\ 
$$



## three functions for comparison


#### function 2, runtime=5.539 s (fastest)
$$
u = Y(Q)-X(Q)\beta \\
K = \sigma^2 \cdot \exp(-\phi \cdot D) + \tau^2 \cdot I_{N1} \\
l = logdet(K) + u^{T} \cdot K^{-1} \cdot u \\
$$

#### function 3, runtime=6.423 s
$$
u = Y(Q)-X(Q)\beta \\
K = \sigma^2 \cdot \exp(-\phi \cdot D) + \tau^2 \cdot I_{N1} \\
L = cholesky(K) \\
l = 2\cdot logdet(L) + u^{T} \cdot K^{-1} \cdot u \\
$$

#### function 1, runtime=6.465 s
$$
u = Y(Q)-X(Q)\beta \\
K = \sigma^2 \cdot \exp(-\phi \cdot D) + \tau^2 \cdot I_{N1} \\
L = cholesky(K) \\
v = L^{-1}\cdot u \\
l = 2\cdot logdet(L) + v^{T}v\\
$$


In [ ]:
#%% log-likelihood function


# function 1, the optimized one, cholesky(K), v=inv(L)u
def lfunction(YQ, XQ, D, beta, var, phi, tau2):
    u = YQ-XQ*beta
    K = var*np.exp(-phi*D) + tau2*np.eye(len(D))
    K = np.matrix(K)
    
    L = np.linalg.cholesky(K)
    sgn, logdetL = slogdet(L)
    v = np.linalg.inv(L)*u
    vv = v.T*v
    
    l = 2*logdetL + vv[0,0]
    return -l


# function 2, direct calculation logdet(K)
def lfunction2(YQ, XQ, D, beta, var, phi, tau2):
    u = YQ-XQ*beta
    K = var*np.exp(-phi*D) + tau2*np.eye(len(D))
    K = np.matrix(K)
    
    sgn, logdetK = slogdet(K)

    vv = u.T*np.linalg.inv(K)*u
    
    l = logdetK + vv[0,0]
    return -l

# function 3, cholesky(K)
def lfunction3(YQ, XQ, D, beta, var, phi, tau2):
    u = YQ-XQ*beta
    K = var*np.exp(-phi*D) + tau2*np.eye(len(D))
    K = np.matrix(K)
    
    L = np.linalg.cholesky(K)
    sgn, logdetL = slogdet(L)
    vv = u.T*np.linalg.inv(K)*u
    
    l = 2*logdetL + vv[0,0]
    return -l

### Setup the variables as function inputs

In [ ]:
#%% setup
YQ = train_y
N1 = YQ.shape[0] # 6297
XQ = np.ones([N1,1])

locs = train_X
D = GetD(locs,locs) # 6297*6297


### parameters initialized
beta = 1.0 # beta = np.mean(YQ) # 0.9344273
var = 4.0 # var = np.std(YQ)**2 # 3.944662179515035
phi = 0.5
tau2 = 1e-4



#%% one-time runtime calculation
time1 = time.time()
l = lfunction(YQ,XQ,D,beta,var,phi,tau2)
time2 = time.time()
print('Fun1 (optimized) time in seconds:', time2-time1)


time1 = time.time()
l = lfunction2(YQ,XQ,D,beta,var,phi,tau2)
time2 = time.time()
print('Fun2 [no v, slogdet(K)] time in seconds:', time2-time1)

time1 = time.time()
l = lfunction3(YQ,XQ,D,beta,var,phi,tau2)
time2 = time.time()
print('Fun3 (no v) time in seconds:', time2-time1)

### calculate 20 times, for each function

In [ ]:
#%% 20-times runtime calculations


betas = np.arange(-1.1,1.1,0.2)
varss = np.arange(1.0,5.0,1)
phis = np.arange(0.1,2.0,0.4)

time1 = time.time()
if True:
    for var in varss:
        for phi in phis:
            l = lfunction(YQ,XQ,D,beta,var,phi,tau2)
time2 = time.time()
print('Fun1 (optimized) time in seconds:', time2-time1)


time1 = time.time()
if True:
    for var in varss:
        for phi in phis:
            l = lfunction2(YQ,XQ,D,beta,var,phi,tau2)
time2 = time.time()
print('Fun2 [no v, slogdet(K)] time in seconds:', time2-time1)

time1 = time.time()
if True:
    for var in varss:
        for phi in phis:
            l = lfunction3(YQ,XQ,D,beta,var,phi,tau2)
time2 = time.time()
print('Fun3 (no v) time in seconds:', time2-time1)
